# CM3070, Final Project

```
University of London
BSc Computer Science
CM3070, Final Project
Hudson Leonardo MENDES
hlm12@student.london.ac.uk
```


## 1. Introduction


### 1.1. Domain-specific Area


### 1.2. Dataset

**Multimodal EmotionLines Dataset(MELD)**[2, 3]

> Multimodal EmotionLines Dataset (MELD) has been created by enhancing and extending EmotionLines dataset. MELD contains the same dialogue instances available in EmotionLines, but it also encompasses audio and visual modality along with text. MELD has more than 1400 dialogues and 13000 utterances from Friends TV series. Multiple speakers participated in the dialogues. Each utterance in a dialogue has been labeled by any of these seven emotions -- Anger, Disgust, Sadness, Joy, Neutral, Surprise and Fear. MELD also has sentiment (positive, negative and neutral) annotation for each utterance.
> (Hakim, 2021)


### 1.3. Objective


## 2. Environment


### 2.1. Dependencies


In [1]:
!cat ../setup.cfg

[metadata]
name = hlm12erc
version = attr: hlm12erc.VERSION
author = Hudson Mendes
author_email = hlm12@student.london.ac.uk
description = Final Project from University of London
long_description = file: README.md, LICENSE
keywords = university-of-london
license = copyright

[options]
zip_safe = False
include_package_data = True
packages = find:
package_dir =
    =src
python_requires = >=3.10
install_requires =
    torch>=2.0.1
    transformers>=4.30.2

[options.package_data]

[options.extras_require]
# development
dev =
    pre-commit>=3.3.3
    black[jupyter]>=23.7.0
    isort>=5.12.0

test =
    pytest>=7.4.0

# mlops
etl =
    kaggle>=1.5.13
    tqdm>=4.65.0
    pandas>=2.0.1
    google-cloud-storage>=2.10.0
    moviepy>=1.0.3
    Pillow>=10.0.0

modeling =
    # no dependencies yet

training =
    # no dependencies yet

serving =
    fire>=0.5.0

all =
    %(dev)s
    %(test)s
    %(etl)s
    %(modeling)s
    %(training)s
    %(serving)s

[options.packages.find]
include =
    ./sr

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging

logging.basicConfig(level=logging.INFO)


### Paths & Locations

In [4]:
import pathlib

dir_data = pathlib.Path("../data")

## 3. Exploratory Data Analysis


### 3.1. Data Extraction, Transformation & Loading


In [5]:
%%capture
%pip install -e '.[etl]'

#### 3.1.1. [E]xtraction


In [6]:
from hlm12erc.etl import ETL, KaggleDataset

ds_kaggle = KaggleDataset(owner="zaber666", name="meld-dataset", subdir="MELD-RAW/MELD.Raw")
etl = ETL(dataset=ds_kaggle).into(uri_or_folderpath=dir_data)

ERROR:tornado.general:SEND Error: Host unreachable
ERROR:tornado.general:SEND Error: Host unreachable
INFO:hlm12erc.etl:Kaggle dataset: zaber666/meld-dataset
INFO:hlm12erc.etl:Workspace set to: /tmp/hlm12erc/etl
INFO:hlm12erc.etl:Extracting dataset into: /tmp/hlm12erc/etl/zaber666-meld-dataset/extracted
INFO:hlm12erc.etl.extraction:Downloading dataset into: /tmp/hlm12erc/etl/zaber666-meld-dataset/extractor/zaber666-meld-dataset


100%|██████████| 11.0G/11.0G [12:37<00:00, 15.6MB/s]
INFO:hlm12erc.etl.extraction:Extracting dataset into: /tmp/hlm12erc/etl/zaber666-meld-dataset/extracted


IsADirectoryError: [Errno 21] Is a directory: '/tmp/hlm12erc/etl/zaber666-meld-dataset/extractor/zaber666-meld-dataset'

In [ ]:
!ls {str(dir_data)}

#### 3.1.2. [T]ransformation (Data Cleaning)


#### 3.1.2. [L]oading


### 3.2. Statistical Analysis


#### 3.2.1. Measures of Spread


#### 3.2.2. Types of Distribution


### 3.3. Data Visualization


#### 3.3.1. Word Cloud & Frequency


#### 3.3.2. Image Grid & Scatter Plot


#### 3.3.3. Audio Feature Plots


## 4. Machine Learning

**Universal Machine-learning Workflow**[1]


### 4.1. Problem Definition


### 4.2. Measure of Success


### 4.3. Evaluation Protocol


### 4.4. Data Preparation


### 4.5. Baseline Model


### 4.6. Overfitting Model


### 4.7. Model Tuning


## 5. Results


## 6. Conclusions


## References

[1] Chollet, François. Deep Learning with Python. Manning, 2017.

[2] Poria, Soujanya, et al. ‘MELD: A Multimodal Multi-Party Dataset for Emotion Recognition in Conversations’. ArXiv [Cs.CL], 2019, http://arxiv.org/abs/1810.02508. arXiv.

[3] Chen, Sheng-Yeh, et al. ‘EmotionLines: An Emotion Corpus of Multi-Party Conversations’. ArXiv [Cs.CL], 2018, http://arxiv.org/abs/1802.08379. arXiv.

[4] Su, Lin, et al. ‘GEM: A General Evaluation Benchmark for Multimodal Tasks’. ArXiv [Cs.CL], 2021, http://arxiv.org/abs/2106.09889. arXiv.
